<a href="https://colab.research.google.com/github/gfeyzakorkmaz/Tries/blob/main/A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
path = '/content/drive/MyDrive/Dataset/complex_marketing_data_enriched.xlsx'

In [5]:
df = pd.read_excel(path)
df.head()

,Customer ID,Age,Gender,Total Spent,Converted,Comments,Purchased Category,Stock,Region,Income Level,...,Campaign,Return Rate (%),Website Visit Frequency,Average Cart Value,Total Spent Diff,Spending Rank,Rolling Return Rate,High Spender,Customer Tenure Quarter,Customer Word Count
0,1,23,F,150,1,Satisfied with the service,Clothing,4,East,Medium,...,Winter Sale,2.981829,3,378.00,NaN,19.0,NaN,No,Q1,0
1,2,35,M,600,1,Needs improvement in delivery,Beauty,86,West,Low,...,Summer Sale,1.833412,6,315.52,450.0,6.0,NaN,Yes,Q3,0
2,3,45,F,800,0,Regular customer,Books,24,North,Medium,...,Black Friday,6.547401,0,250.69,200.0,1.0,3.787547,Yes,Q3,1
3,4,52,M,200,0,Occasional buyer,Beauty,10,East,Medium,...,Winter Sale,3.923717,1,222.58,-600.0,18.0,4.101510,No,Q2,0
4,5,29,F,300,1,High spender,Books,47,East,Medium,...,Winter Sale,2.306644,4,298.68,100.0,15.0,4.259254,No,Q3,0


In [8]:
top_categories = df.groupby('Purchased Category')['Total Spent'].sum().sort_values(ascending=False).head(5)
print(top_categories)

Purchased Category
Books          2350
Home Goods     2090
Electronics    1680
Clothing       1600
Beauty         1200
Name: Total Spent, dtype: int64


In [9]:
avg_spending_by_gender = df.groupby('Gender')['Total Spent'].mean()
print(avg_spending_by_gender)

Gender
F    467.272727
M    420.000000
Name: Total Spent, dtype: float64


In [10]:
top_region_by_conversions = df[df['Converted'] == 1].groupby('Region').size().idxmax()
print(top_region_by_conversions)

East


In [11]:
discount_conversion_corr = df[['Discount Availed', 'Converted']].corr()
print(discount_conversion_corr)

                  Discount Availed  Converted
Discount Availed          1.000000   0.093659
Converted                 0.093659   1.000000


In [12]:
high_tenure_low_spending = df[(df['Customer Tenure (Years)'] > 10) & (df['Total Spent'] < 300)]
print(high_tenure_low_spending )

Empty DataFrame
Columns: [Customer ID, Age, Gender, Total Spent, Converted, Comments, Purchased Category, Stock, Region, Income Level, Discount Availed, Customer Tenure (Years), Campaign, Return Rate (%), Website Visit Frequency, Average Cart Value, Total Spent Diff, Spending Rank, Rolling Return Rate, High Spender, Customer Tenure Quarter, Customer Word Count]
Index: []

[0 rows x 22 columns]


In [13]:
pivot_campaign_region = df.pivot_table(
    values='Total Spent',
    index='Campaign',
    columns='Region',
    aggfunc='mean',
    fill_value=0
)
print(pivot_campaign_region)

Region         East  North   West
Campaign                         
Black Friday  305.0  800.0  550.0
Summer Sale     0.0    0.0  425.0
Winter Sale   360.0  380.0    0.0


In [15]:
return_rate_category = df.groupby('Purchased Category').apply(
    lambda x: (x['Return Rate (%)'] * x['Total Spent']).sum() / x['Total Spent'].sum()
)
print(return_rate_category)

Purchased Category
Beauty         3.099215
Books          5.013504
Clothing       5.283975
Electronics    4.651508
Home Goods     4.678345
dtype: float64


<ipython-input-15-cde7bc0e0a25>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return_rate_category = df.groupby('Purchased Category').apply(


In [16]:
high_visitors = df[df['Website Visit Frequency'] > df['Website Visit Frequency'].mean()]
print(high_visitors)

    Customer ID  Age Gender  Total Spent  Converted  \
1             2   35      M          600          1   
4             5   29      F          300          1   
7             8   27      M          120          1   
9            10   33      M          750          1   
12           13   48      F          400          1   
13           14   32      M          320          0   
17           18   35      M          700          1   
18           19   50      F          250          0   

                           Comments Purchased Category  Stock Region  \
1     Needs improvement in delivery             Beauty     86   West   
4                      High spender              Books     47   East   
7                      New customer         Home Goods      0  North   
9              High conversion rate           Clothing     45  South   
12                 Frequent shopper             Beauty      0   East   
13             Unhappy with returns        Electronics     85   East   


In [17]:
top_campaign_cart_value = df.groupby('Campaign')['Average Cart Value'].mean().idxmax()
print(top_campaign_cart_value)

Summer Sale


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


features = ['Age', 'Total Spent', 'Discount Availed', 'Customer Tenure (Years)']
X = df[features]
y = df['Converted']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logistic_model = LogisticRegression(max_iter=200)
logistic_model.fit(X_train, y_train)

y_pred = logistic_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(accuracy)

0.5
